1.regression model

1.1 regression is a statistic method to solve problem and proceeds a model between one or more independent variable and dependent variable
回归是对一个或多个自变量和因变量之间的关系进行建模，求解的一种统计方法

In [ ]:
# import some packages
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np